In [1]:
# --- import libraries
import os
import h5py
import matplotlib as mpl
import matplotlib.pyplot as plt
os.environ["PROJ_LIB"] = "C:/Users/admin/Anaconda3/Library/share"
from mpl_toolkits.basemap import Basemap
import numpy as np

In [2]:
# --- path
path = '../'
inp_dir = path + 'inp_data/'
out_dir = path + 'out_data/'
mid_dir = path + 'mid_dir/'
plt_dir = path + 'plots/'

In [3]:
# === read all datasets in file and test for group
def read_h5_info(filename):

    # --- check file structure
    def traverse_datasets(hdf_file):
        def h5py_dataset_iterator(g, prefix=''):
            for key in g.keys():
                item = g[key]
                path = f'{prefix}/{key}'
                if isinstance(item, h5py.Dataset):  # test for dataset
                    yield (path, item)
                elif isinstance(item, h5py.Group):  # test for group (go down)
                    yield from h5py_dataset_iterator(item, path)

        with h5py.File(hdf_file, 'r') as f:
            for path, _ in h5py_dataset_iterator(f):
                yield path

    # --- open
    f = h5py.File(filename, 'r')

    # --- read structure
    for dset in traverse_datasets(filename):
        print('Path:', dset)
        print('Shape:', f[dset].shape)
        print('Data type:', f[dset].dtype)

In [4]:
def save_plot(path, ext='png', close=True, verbose=True):
    import matplotlib.pyplot as plt
    import os

    """Save a figure from pyplot.
    Parameters
    ----------
    path : string
        The path (and filename, without the extension) to save the
        figure to.
    ext : string (default='png')
        The file extension. This must be supported by the active
        matplotlib backend (see matplotlib.backends module).  Most
        backends support 'png', 'pdf', 'ps', 'eps', and 'svg'.
    close : boolean (default=True)
        Whether to close the figure after saving.  If you want to save
        the figure multiple times (e.g., to multiple formats), you
        should NOT close it in between saves or you will have to
        re-plot it.
    verbose : boolean (default=True)
        Whether to print information about when and where the image
        has been saved.
    """
    # --- Extract the directory and filename from the given path
    directory = os.path.split(path)[0]
    filename = "%s.%s"%(os.path.split(path)[1], ext)
    if directory == '':
        directory = '.'

    # --- If the directory does not exist, create it
    if not os.path.exists(directory):
        os.makedirs(directory)

    # --- The final path to save to
    savepath = os.path.join(directory, filename)

    if verbose:
        print('        Saving figure to           : ', savepath)

    # --- Actually save the figure
    plt.savefig(savepath, dpi=600, bbox_inches='tight')

    # --- Close it
    if close:
        plt.close()

    if verbose:
        print("plot_save - OK")

In [5]:
fname = inp_dir + 'MOP02J-20150101-L2V16.2.3.he5'

In [6]:
#read_h5_info(fname)

In [12]:
with h5py.File(fname, mode='r') as f:
    
    name = '/HDFEOS/SWATHS/MOP02/Data Fields/RetrievedCOTotalColumn'
    data = f[name][:, 0]
    
    # units attribute is an array of string.
    #units = f[name].attrs['units'][0]
    fillvalue = f[name].attrs['_FillValue']

    data[data == fillvalue] = np.nan

    # Get the geolocation data
    latitude = f['/HDFEOS/SWATHS/MOP02/Geolocation Fields/Latitude'][:]
    longitude = f['/HDFEOS/SWATHS/MOP02/Geolocation Fields/Longitude'][:]

    m = Basemap(projection='cyl', resolution='l',
                 llcrnrlat=-90, urcrnrlat=90,
                 llcrnrlon=-180, urcrnrlon=180)
#                llcrnrlat=40, urcrnrlat=70,
#                llcrnrlon=70, urcrnrlon=110)    
#                llcrnrlat=20, urcrnrlat=50,
#                llcrnrlon=100, urcrnrlon=160)    
    m.drawcoastlines(linewidth=0.5)
    m.drawparallels(np.arange(-90, 91, 30))
    m.drawmeridians(np.arange(-180, 180, 30), labels=[True,False,False,True])
    sc = m.scatter(longitude, latitude, c=data, s=1, cmap=plt.cm.jet,
                   edgecolors=None, linewidth=0)
    cb = m.colorbar()
    #cb.set_label(units)

    basename = os.path.basename(fname)
    plt.title('{0}\n{1}'.format(basename, 'RetrievedCOTotalColumn'))
    
    # --- save to plot
    plot_name = plt_dir + basename
    save_plot(plot_name, ext="png", close=True, verbose=False)
    #plt.show()